In [134]:
# Initialization
!pip install playwright
!playwright install

import os
import json
from playwright.async_api import async_playwright

^C


In [135]:
# input password and username here; do not share this with anyone!
username = INSERT_USERNAME
password = INSERT_PASSWORD

# scrape for the first time
async def new_scrape(url):
    async with async_playwright() as playwright:

        # Initialization
        browser = await playwright.chromium.launch(headless=True) 
        page = await browser.new_page()

        # navigate to the Quest webpage
        await page.goto(url)

        # entering username/email
        await page.keyboard.type(username)
        await page.keyboard.press('Enter')

        # entering password
        await page.keyboard.type(password)
        await page.keyboard.press('Enter')

        # find and print the verification code if on a new device with no cookies
        try:
            await page.wait_for_selector('div.verification-code',timeout=10000)
            code = await page.inner_text('div.verification-code')
            print(code) # code needs to be entered in Duo Mobile


            # wait for 2 factor authentication to pass
            await page.wait_for_selector('#trust-browser-button',timeout=60000)
    
            # Accept device as own
            await page.keyboard.press('Tab')
            await page.keyboard.press('Enter')
            
        finally:

            # Nagivate to the funds tab
            await page.wait_for_selector('header', state='visible')
            await page.keyboard.press('Tab')
            await page.keyboard.press('Tab')
            await page.keyboard.press('Tab')
            await page.keyboard.press('Tab')
            await page.keyboard.press('Enter')
    
            # Select the flex dollars amount
            await page.wait_for_selector('td.text-right')
            data = await page.inner_text('td.text-right')
    
    
            # Get all cookies
            cookies = await page.context.cookies()
    
            # Save cookies to a file
            with open('cookies.json', 'w') as cookie_file:
                json.dump(cookies, cookie_file)
            
            # close the browser
            await browser.close()
            
        return data
        
# scrape with cookies
async def scrape(url):
    async with async_playwright() as playwright:
        # Launch the browser
        browser = await playwright.chromium.launch(headless=True)
        context = await browser.new_context()

        # Load cookies from file
        with open('cookies.json', 'r') as cookie_file:
            cookies = json.load(cookie_file)

        # set cookies in the browser 
        await context.add_cookies(cookies)

        # Navigate to the page with the cookies
        page = await context.new_page()
        await page.goto(url)

        # Nagivate to the funds tab
        await page.wait_for_selector('header', state='visible')
        await page.keyboard.press('Tab')
        await page.keyboard.press('Tab')
        await page.keyboard.press('Tab')
        await page.keyboard.press('Tab')
        await page.keyboard.press('Enter')
    
        # Select the flex dollars amount
        await page.wait_for_selector('td.text-right')
        data = await page.inner_text('td.text-right')
    
    
        # Get all cookies
        cookies = await page.context.cookies()
    
        # Save cookies to a file
        with open('cookies.json', 'w') as cookie_file:
            json.dump(cookies, cookie_file)
            
        # close the browser
        await browser.close()
        return data

# use scrape or new_scrape depending if cookies.json exists already
if os.path.isfile('cookies.json'):
    balance = await scrape('https://secure.touchnet.net/C22566_oneweb/')
    
else:
    balance = await new_scrape('https://secure.touchnet.net/C22566_oneweb/')


# output
print('Your total Flex Dollar balance is: ', balance , sep="")

Your total Flex Dollar balance is: $15.35
